# Podział sekretu

Autorzy

- Jakub Błażejowski | 145393

In [ ]:
# requires-python = ">3.8.0"

# python requirements

"""
numpy==1.24.1
sympy==1.11.1
"""

import math
from typing import List
import numpy
from sympy import randprime
from IPython.display import Markdown
from ipywidgets import Box, Button, Checkbox, interactive_output, IntText, Output, Text

In [ ]:
%%html
<style>
    .output {
        align-items: center;
    }
    div.output_subarea {
        max-width: none;
        display: flex;
        justify-content: center;
    }
    :root {
        --jp-widgets-inline-label-width: 200px;
        --jp-widgets-inline-width: 408px;
    }
</style>

In [ ]:
class Encryptor:
    def __init__(
        self,
        shares_number_widget: IntText,
        required_shares_number_widget: IntText,
        secret_widget: IntText,
        prime_widget: IntText,
        generate_prime_widget: Checkbox,
        start_widget: Button,
        output_widget: Output
    ):
        self.shares_number_widget = shares_number_widget
        self.required_shares_number_widget = required_shares_number_widget
        self.secret_widget = secret_widget
        self.prime_widget = prime_widget
        self.generate_prime_widget = generate_prime_widget
        self.start_widget = start_widget
        self.output_widget = output_widget
        self.shares = []
        
        self.widgets = [
            shares_number_widget,
            required_shares_number_widget,
            secret_widget,
            prime_widget,
            generate_prime_widget,
            start_widget
        ]
        
        self.encrypt = output_widget.capture()(self.encrypt)
        
    def disable_widgets(self):
        for widget in self.widgets:
            widget.enabled = False
            
    def enable_widgets(self):
        for widget in self.widgets:
            widget.enabled = True

    def encrypt(self):
        self.output_widget.clear_output()
        self.disable_widgets()
        self.start_widget.description = "Computing"
        self.shares = []

        shares_number = self.shares_number_widget.value
        required_shares_number = self.required_shares_number_widget.value
        secret = self.secret_widget.value
        prime = prime_widget.value
        
        width = math.floor(math.log(shares_number, 10)) + 1
        
        if self.generate_prime_widget.value:
            start = max(shares_number, secret) + 1
            prime = randprime(start, max(2 * start, 1000000))
            self.prime_widget.value = prime
            
        rng = numpy.random.default_rng()
            
        a = rng.integers(1, prime, size=required_shares_number - 1)
        print("coefficients: ", a)
        
        for i in range(1, shares_number + 1):
            x = numpy.full(required_shares_number - 1, i)**numpy.arange(1, required_shares_number)
            share = (secret + (a*x).sum())%prime
                
            print(f"share no. {i:{width}}: ", share)
            self.shares.append(share)

        self.start_widget.description = "Start"
        self.enable_widgets()

In [ ]:
class Decryptor:
    def __init__(
        self,
        prime_widget: IntText,
        share_index_widgets: List[IntText],
        share_widgets: List[IntText],
        start_widget: Button,
        output_widget: Output
    ):
        self.prime_widget = prime_widget
        self.share_index_widgets = share_index_widgets
        self.share_widgets = share_widgets
        self.start_widget = start_widget
        self.output_widget = output_widget
        
        self.widgets = share_index_widgets + share_widgets
        self.widgets.append(start_widget)
        self.widgets.append(prime_widget)
        
        self.decrypt = output_widget.capture()(self.decrypt)
        
    def disable_widgets(self):
        for widget in self.widgets:
            widget.enabled = False
            
    def enable_widgets(self):
        for widget in self.widgets:
            widget.enabled = True

    def decrypt(self):
        self.output_widget.clear_output()
        self.disable_widgets()
        self.start_widget.description = "Computing"
        
        prime = self.prime_widget.value
        share_indexes = numpy.array([x.value for x in self.share_index_widgets])
        shares = numpy.array([x.value for x in self.share_widgets])
        
        a = numpy.zeros((share_indexes.shape[0], share_indexes.shape[0]))
        a[:, :] = share_indexes
        a = a.transpose()
        a = a**numpy.arange(0, share_indexes.shape[0])
        print("a & b matrices:")
        print(a, shares)
        
        secret = round(numpy.linalg.lstsq(a, shares, rcond=-1)[0][0])%prime
        print("secret: ", secret)

        self.start_widget.description = "Start"
        self.enable_widgets()

In [ ]:
shares_number_widget = IntText(value=4, description="Number of shares:")
required_shares_number_widget = IntText(value=3, description="Required number of shares:")
secret_widget = IntText(value=954, description="Secret:")
prime_widget = IntText(value=1523, description="Prime:")
generate_prime_widget = Checkbox(description="Genereate prime:")
start_widget = Button(description="Start")
output_widget = Output()

encryptor = Encryptor(
    shares_number_widget,
    required_shares_number_widget,
    secret_widget,
    prime_widget,
    generate_prime_widget,
    start_widget,
    output_widget
)

start_widget.on_click(lambda _: encryptor.encrypt())

display(
    shares_number_widget,
    required_shares_number_widget,
    secret_widget,
    prime_widget,
    generate_prime_widget,
    start_widget,
    output_widget
);

In [ ]:
shares_number_widget2 = IntText(
    value=required_shares_number_widget.value,
    description="Number of shares:"
)

prime_widget2 = IntText(
    value=prime_widget.value,
    description="Prime:"
)

output_widget2 = Output()
decryptor = Decryptor(prime_widget2, [], [], None, output_widget2)

def run(shares_number: int):
    share_index_widgets = [IntText(value=i, description="Index:") for i in range(1, shares_number + 1)]
    share_widgets = [IntText(description="Share:") for _ in range(1, shares_number + 1)]
    
    for i, share in zip(range(len(share_widgets)), encryptor.shares):
        share_widgets[i].value = encryptor.shares[i]
        
    shares_combined = [Box([i, j]) for i, j in zip(share_index_widgets, share_widgets)]
    start_widget = Button(description="Start")
    
    decryptor.share_index_widgets = share_index_widgets
    decryptor.share_widgets = share_widgets
    decryptor.start_widget = start_widget
    decryptor.widgets = share_index_widgets + share_widgets
    decryptor.widgets.append(prime_widget2)
    decryptor.widgets.append(start_widget)
    
    start_widget.on_click(lambda _: decryptor.decrypt())
    
    display(
        *shares_combined,
        start_widget
    )

interactive_output_widget = interactive_output(run, {"shares_number": shares_number_widget2})
display(shares_number_widget2, prime_widget2, interactive_output_widget, output_widget2);